In [ ]:
# pip install pandas openpyxl

第一版

In [ ]:
import requests
import json
import pandas as pd

# 指定 ISIN 程式碼和 URL
# https://api.boerse-frankfurt.de/v1/data/price_information?isin=US872898AJ06&mic=XFRA
_ISIN_CODE = 'US872898AJ06'
url = f"https://api.boerse-frankfurt.de/v1/data/price_information?isin={_ISIN_CODE}&mic=XFRA"

# 發送請求並取得響應
response = requests.get(url)
#
print(response.text)

# 檢查響應狀態碼
if response.status_code == 200:
    # 將回應內容按照 "data:" 進行分割
    parts = response.text.split("data:")

    # 檢查是否存在至少兩個部分
    if len(parts) > 1:
        # 嘗試解析第一部分的 JSON 數據
        try:
            data_json = parts[1].strip()
            data = json.loads(data_json)

            # 轉換為 pandas DataFrame
            df = pd.DataFrame([data])

            # 儲存為 Excel 文件
            excel_file = 'price_information.xlsx'
            df.to_excel(excel_file, index=False)

            print(f"資料已儲存為 {excel_file}")

        except json.JSONDecodeError as e:
            print(f"JSON 無法解析： {e}")
    else:
        print("未找到有效的 'data:' 資料")
else:
    print(f"發生錯誤：無法取得資料：{response.status_code}")

#
df

第二版

In [ ]:
import requests
import json
import pandas as pd

# 指定 ISIN 程式碼和 URL
_ISIN_CODE = 'US872898AJ06'
url = f"https://api.boerse-frankfurt.de/v1/data/price_information?isin={_ISIN_CODE}&mic=XFRA"

# 發送請求並取得響應
with requests.get(url, stream=True) as response:
    # 確保響應成功
    if response.status_code == 200:
        data_count = 0
        data_json = None
        for line in response.iter_lines():
            # 解碼每一行
            line = line.decode('utf-8')

            # 檢查行是否包含 'data:'
            if 'data:' in line:
                data_count += 1
                if data_count == 1:
                    # 提取第一次出現 'data:' 之後的內容
                    data_json = line.split('data:', 1)[1].strip()
                elif data_count == 2:
                    # 當第二次出現 'data:' 時終止循環
                    break
    else:
        print(f"發生錯誤：無法取得資料：{response.status_code}")

# 檢查是否有取得到數據
if data_json:
    try:
        # 解析 JSON 數據
        data = json.loads(data_json)

        # 轉換為 pandas DataFrame
        df = pd.DataFrame([data])

        # 儲存為 Excel 文件
        excel_file = 'price_information.xlsx'
        df.to_excel(excel_file, index=False)

        print(f"資料已儲存為 {excel_file}")
    except json.JSONDecodeError as e:
        print(f"JSON 無法解析： {e}")
else:
    print("未找到有效的 'data:' 資料")


第三版：改用 with

In [ ]:
import requests
import json
import pandas as pd

# 指定 ISIN 程式碼和 URL
_ISIN_CODE = 'US872898AJ06'
url = f"https://api.boerse-frankfurt.de/v1/data/price_information?isin={_ISIN_CODE}&mic=XFRA"

# 使用 with 語句發送請求並取得響應
with requests.get(url, stream=True) as response:
    # 確保響應成功
    if response.status_code == 200:
        data_count = 0
        data_json = None
        for line in response.iter_lines():
            # 解碼每一行
            line = line.decode('utf-8')
            # 檢查行是否包含 'data:'
            if 'data:' in line:
                data_count += 1
                if data_count == 1:
                    # 提取第一次出現 'data:' 之後的內容
                    data_json = line.split('data:', 1)[1].strip()
                elif data_count == 2:
                    # 當第二次出現 'data:' 時終止循環
                    break
    else:
        print(f"發生錯誤：無法取得資料：{response.status_code}")

# 檢查是否有取得到數據
if data_json:
    try:
        # 解析 JSON 數據
        data = json.loads(data_json)

        # 轉換為 pandas DataFrame
        df = pd.DataFrame([data])

        # 儲存為 Excel 文件
        excel_file = 'price_information.xlsx'
        df.to_excel(excel_file, index=False)

        print(f"資料已儲存為 {excel_file}")
    except json.JSONDecodeError as e:
        print(f"JSON 無法解析： {e}")
else:
    print("未找到有效的 'data:' 資料")

df

取出欄位

In [ ]:
# 取出所有欄位名稱並輸出
column_names = df.columns.tolist()
print("欄位名稱:", column_names)

映射中文欄位名稱並轉換時間戳

轉換時間戳：pip install pytz

In [19]:
import requests
import json
import pandas as pd
import pytz

# 指定 ISIN 程式碼和 URL
_ISIN_CODE = 'US872898AJ06'
url = f"https://api.boerse-frankfurt.de/v1/data/price_information?isin={_ISIN_CODE}&mic=XFRA"

# 定義英文欄位名稱到中文的映射
column_mapping = {
    'isin': '證券編碼',
    'lastPrice': '最新價格',
    'timestampLastPrice': '最新價格時間戳',
    'changeToPrevDayAbsolute': '今日漲跌價格',
    'changeToPrevDayInPercent': '今日漲跌幅度',
    'closingPricePrevTradingDay': '前日收盤價',
    'mic': '市場識別碼',
    'dayHigh': '最高價',
    'dayLow': '最低價',
    'priceFixings': '價格定點',
    'tradedInPercent': '以百分比交易',
    'tradingTimeEnd': '交易結束時間',
    'tradingTimeStart': '交易開始時間',
    'turnoverInEur': '成交額（歐元）',
    'turnoverInPieces': '成交量（件數）',
    'turnoverNominal': '名義成交額',
    'weeks52High': '52週最高價',
    'weeks52Low': '52週最低價',
    'currency': '貨幣',
    'minimumTradableUnit': '最小可交易單位'
}

# 使用 with 語句發送請求並取得響應
with requests.get(url, stream=True) as response:
    # 確保響應成功
    if response.status_code == 200:
        data_count = 0
        data_json = None
        for line in response.iter_lines():
            # 解碼每一行
            line = line.decode('utf-8')
            # 檢查行是否包含 'data:'
            if 'data:' in line:
                data_count += 1
                if data_count == 1:
                    # 提取第一次出現 'data:' 之後的內容
                    data_json = line.split('data:', 1)[1].strip()
                elif data_count == 2:
                    # 當第二次出現 'data:' 時終止循環
                    break
    else:
        print(f"發生錯誤：無法取得資料：{response.status_code}")

# 檢查是否有取得到數據
if data_json:
    try:
        # 解析 JSON 數據
        data = json.loads(data_json)

        # 轉換為 pandas DataFrame
        df = pd.DataFrame([data])
        # 轉換 DataFrame 的列名
        df.rename(columns=column_mapping, inplace=True)

        # 將 '最新價格時間戳' 轉換為台灣時間 (UTC+8) 並格式化
        df['最新價格時間戳'] = pd.to_datetime(df['最新價格時間戳'])
        taipei_tz = pytz.timezone('Asia/Taipei')
        df['最新價格時間戳'] = df['最新價格時間戳'].dt.tz_convert(taipei_tz)
        df['最新價格時間戳'] = df['最新價格時間戳'].dt.strftime('%Y-%m-%d %H:%M')
        
        # 儲存為 Excel 文件
        excel_file = 'price_information.xlsx'
        df.to_excel(excel_file, index=False)

        print(f"資料已儲存為 {excel_file}")
    except json.JSONDecodeError as e:
        print(f"JSON 無法解析： {e}")
else:
    print("未找到有效的 'data:' 資料")

df

資料已儲存為 price_information.xlsx


,證券編碼,最新價格,最新價格時間戳,今日漲跌價格,今日漲跌幅度,前日收盤價,市場識別碼,最高價,最低價,價格定點,以百分比交易,交易結束時間,交易開始時間,成交額（歐元）,成交量（件數）,名義成交額,52週最高價,52週最低價,貨幣,最小可交易單位
0,US872898AJ06,97.17,2023-12-14 16:44,2.7,2.85805,94.47,XFRA,97.17,97.17,None,True,17:30:00,08:00:00,0.0,None,0.0,101.39,80.95,"{'originalValue': 'USD', 'translations': {'de'...",200000


進一步將交易的開始與結束時間也轉換為台灣時間，依據原始數據判斷原本的時區是CET/CEST 

In [22]:
import requests
import json
import pandas as pd
import pytz

# 指定 ISIN 程式碼和 URL
_ISIN_CODE = 'US872898AJ06'
url = f"https://api.boerse-frankfurt.de/v1/data/price_information?isin={_ISIN_CODE}&mic=XFRA"

# 定義英文欄位名稱到中文的映射
column_mapping = {
    'isin': '證券編碼',
    'lastPrice': '最新價格',
    'timestampLastPrice': '最新價格時間戳',
    'changeToPrevDayAbsolute': '今日漲跌價格',
    'changeToPrevDayInPercent': '今日漲跌幅度',
    'closingPricePrevTradingDay': '前日收盤價',
    'mic': '市場識別碼',
    'dayHigh': '最高價',
    'dayLow': '最低價',
    'priceFixings': '價格定點',
    'tradedInPercent': '以百分比交易',
    'tradingTimeEnd': '交易結束時間',
    'tradingTimeStart': '交易開始時間',
    'turnoverInEur': '成交額（歐元）',
    'turnoverInPieces': '成交量（件數）',
    'turnoverNominal': '名義成交額',
    'weeks52High': '52週最高價',
    'weeks52Low': '52週最低價',
    'currency': '貨幣',
    'minimumTradableUnit': '最小可交易單位'
}

# 使用 with 語句發送請求並取得響應
with requests.get(url, stream=True) as response:
    # 確保響應成功
    if response.status_code == 200:
        data_count = 0
        data_json = None
        for line in response.iter_lines():
            # 解碼每一行
            line = line.decode('utf-8')
            # 檢查行是否包含 'data:'
            if 'data:' in line:
                data_count += 1
                if data_count == 1:
                    # 提取第一次出現 'data:' 之後的內容
                    data_json = line.split('data:', 1)[1].strip()
                elif data_count == 2:
                    # 當第二次出現 'data:' 時終止循環
                    break
    else:
        print(f"發生錯誤：無法取得資料：{response.status_code}")

# 檢查是否有取得到數據
if data_json:
    try:
        # 解析 JSON 數據
        data = json.loads(data_json)

        # 轉換為 pandas DataFrame
        df = pd.DataFrame([data])
        # 轉換 DataFrame 的列名
        df.rename(columns=column_mapping, inplace=True)

        # 將 '最新價格時間戳' 轉換為台灣時間 (UTC+8) 並格式化
        df['最新價格時間戳'] = pd.to_datetime(df['最新價格時間戳'])
        taipei_tz = pytz.timezone('Asia/Taipei')
        df['最新價格時間戳'] = df['最新價格時間戳'].dt.tz_convert(taipei_tz)
        df['最新價格時間戳'] = df['最新價格時間戳'].dt.strftime('%Y-%m-%d %H:%M')
        
        # 原始數據為中歐時間（CET/CEST）
        frankfurt_tz = pytz.timezone('Europe/Berlin')
        # 將 '交易開始時間' 和 '交易結束時間' 從字串轉換為 datetime 對象
        # 並轉換為當地時間
        df['交易開始時間'] = pd.to_datetime(df['交易開始時間']).dt.tz_localize(frankfurt_tz)
        df['交易結束時間'] = pd.to_datetime(df['交易結束時間']).dt.tz_localize(frankfurt_tz)
        # 轉換為台灣時間 (UTC+8)
        df['交易開始時間'] = df['交易開始時間'].dt.tz_convert(taipei_tz).dt.strftime('%Y-%m-%d %H:%M')
        df['交易結束時間'] = df['交易結束時間'].dt.tz_convert(taipei_tz).dt.strftime('%Y-%m-%d %H:%M')

        # 儲存為 Excel 文件
        excel_file = 'price_information.xlsx'
        df.to_excel(excel_file, index=False)

        print(f"資料已儲存為 {excel_file}")
    except json.JSONDecodeError as e:
        print(f"JSON 無法解析： {e}")
else:
    print("未找到有效的 'data:' 資料")

df

資料已儲存為 price_information.xlsx


,證券編碼,最新價格,最新價格時間戳,今日漲跌價格,今日漲跌幅度,前日收盤價,市場識別碼,最高價,最低價,價格定點,以百分比交易,交易結束時間,交易開始時間,成交額（歐元）,成交量（件數）,名義成交額,52週最高價,52週最低價,貨幣,最小可交易單位
0,US872898AJ06,97.17,2023-12-14 16:44,2.7,2.85805,94.47,XFRA,97.17,97.17,None,True,2023-12-16 00:30,2023-12-15 15:00,0.0,None,0.0,101.39,80.95,"{'originalValue': 'USD', 'translations': {'de'...",200000


多檔債券

In [1]:
import requests
import json
import pandas as pd
import pytz

# 定義要查詢的 ISIN 程式碼列表
isin_codes = [
    'US872898AJ06',
    'USF2893TAE67',
    'US02209SBE28',
    'US02209SBF92',
    'US05526DBV64',
    'US30303M8J41'
]

# 定義欄位名稱映射
column_mapping = {
    'isin': '證券編碼',
    'lastPrice': '最新價格',
    'timestampLastPrice': '最新價格時間戳',
    'changeToPrevDayAbsolute': '今日漲跌價格',
    'changeToPrevDayInPercent': '今日漲跌幅度',
    'closingPricePrevTradingDay': '前日收盤價',
    'mic': '市場識別碼',
    'dayHigh': '最高價',
    'dayLow': '最低價',
    'priceFixings': '價格定點',
    'tradedInPercent': '以百分比交易',
    'tradingTimeEnd': '交易結束時間',
    'tradingTimeStart': '交易開始時間',
    'turnoverInEur': '成交額（歐元）',
    'turnoverInPieces': '成交量（件數）',
    'turnoverNominal': '名義成交額',
    'weeks52High': '52週最高價',
    'weeks52Low': '52週最低價',
    'currency': '貨幣',
    'minimumTradableUnit': '最小可交易單位'
}

# 定義一個空的 DataFrame 來儲存所有商品的數據
all_data = pd.DataFrame()

# 遍歷 ISIN 程式碼列表
for _ISIN_CODE in isin_codes:
    url = f"https://api.boerse-frankfurt.de/v1/data/price_information?isin={_ISIN_CODE}&mic=XFRA"

    with requests.get(url, stream=True) as response:
        if response.status_code == 200:
            data_count = 0
            data_json = None
            for line in response.iter_lines():
                line = line.decode('utf-8')
                if 'data:' in line:
                    data_count += 1
                    if data_count == 1:
                        data_json = line.split('data:', 1)[1].strip()
                    elif data_count == 2:
                        break
        else:
            print(f"發生錯誤：無法取得 {_ISIN_CODE} 的資料：{response.status_code}")
            continue

    if data_json:
        try:
            data = json.loads(data_json)
            df = pd.DataFrame([data])
            df.rename(columns=column_mapping, inplace=True)

            # 處理時間轉換等
            # 將 '最新價格時間戳' 轉換為台灣時間 (UTC+8) 並格式化
            df['最新價格時間戳'] = pd.to_datetime(df['最新價格時間戳'])
            taipei_tz = pytz.timezone('Asia/Taipei')
            df['最新價格時間戳'] = df['最新價格時間戳'].dt.tz_convert(taipei_tz)
            df['最新價格時間戳'] = df['最新價格時間戳'].dt.strftime('%Y-%m-%d %H:%M')
            
            # 原始數據為中歐時間（CET/CEST）
            frankfurt_tz = pytz.timezone('Europe/Berlin')
            # 將 '交易開始時間' 和 '交易結束時間' 從字串轉換為 datetime 對象
            # 並轉換為當地時間
            df['交易開始時間'] = pd.to_datetime(df['交易開始時間']).dt.tz_localize(frankfurt_tz)
            df['交易結束時間'] = pd.to_datetime(df['交易結束時間']).dt.tz_localize(frankfurt_tz)
            # 轉換為台灣時間 (UTC+8)
            df['交易開始時間'] = df['交易開始時間'].dt.tz_convert(taipei_tz).dt.strftime('%Y-%m-%d %H:%M')
            df['交易結束時間'] = df['交易結束時間'].dt.tz_convert(taipei_tz).dt.strftime('%Y-%m-%d %H:%M')
            # 將這個商品的數據添加到總的 DataFrame 中
            all_data = pd.concat([all_data, df])
        except json.JSONDecodeError as e:
            print(f"JSON 無法解析 {_ISIN_CODE} 的數據： {e}")

# 儲存所有商品的數據到 Excel 文件
excel_file = 'all_price_information.xlsx'
all_data.to_excel(excel_file, index=False)
print(f"所有商品的資料已儲存為 {excel_file}")


所有商品的資料已儲存為 all_price_information.xlsx


將資料添加在原有資料之後，不是覆蓋

In [3]:
import requests
import json
import pandas as pd
import pytz
import os

# 定義要查詢的 ISIN 程式碼列表
isin_codes = [
    'US872898AJ06',
    'USF2893TAE67',
    'US02209SBE28',
    'US02209SBF92',
    'US05526DBV64',
    'US30303M8J41'
]

# 定義欄位名稱映射
column_mapping = {
    'isin': '證券編碼',
    'lastPrice': '最新價格',
    'timestampLastPrice': '最新價格時間戳',
    'changeToPrevDayAbsolute': '今日漲跌價格',
    'changeToPrevDayInPercent': '今日漲跌幅度',
    'closingPricePrevTradingDay': '前日收盤價',
    'mic': '市場識別碼',
    'dayHigh': '最高價',
    'dayLow': '最低價',
    'priceFixings': '價格定點',
    'tradedInPercent': '以百分比交易',
    'tradingTimeEnd': '交易結束時間',
    'tradingTimeStart': '交易開始時間',
    'turnoverInEur': '成交額（歐元）',
    'turnoverInPieces': '成交量（件數）',
    'turnoverNominal': '名義成交額',
    'weeks52High': '52週最高價',
    'weeks52Low': '52週最低價',
    'currency': '貨幣',
    'minimumTradableUnit': '最小可交易單位'
}

# 定義一個空的 DataFrame 來儲存所有商品的數據
all_data = pd.DataFrame()

# 遍歷 ISIN 程式碼列表
for _ISIN_CODE in isin_codes:
    url = f"https://api.boerse-frankfurt.de/v1/data/price_information?isin={_ISIN_CODE}&mic=XFRA"

    with requests.get(url, stream=True) as response:
        if response.status_code == 200:
            data_count = 0
            data_json = None
            for line in response.iter_lines():
                line = line.decode('utf-8')
                if 'data:' in line:
                    data_count += 1
                    if data_count == 1:
                        data_json = line.split('data:', 1)[1].strip()
                    elif data_count == 2:
                        break
        else:
            print(f"發生錯誤：無法取得 {_ISIN_CODE} 的資料：{response.status_code}")
            continue

    if data_json:
        try:
            data = json.loads(data_json)
            df = pd.DataFrame([data])
            df.rename(columns=column_mapping, inplace=True)

            # 處理時間轉換等
            # 將 '最新價格時間戳' 轉換為台灣時間 (UTC+8) 並格式化
            df['最新價格時間戳'] = pd.to_datetime(df['最新價格時間戳'])
            taipei_tz = pytz.timezone('Asia/Taipei')
            df['最新價格時間戳'] = df['最新價格時間戳'].dt.tz_convert(taipei_tz)
            df['最新價格時間戳'] = df['最新價格時間戳'].dt.strftime('%Y-%m-%d %H:%M')
            
            # 原始數據為中歐時間（CET/CEST）
            frankfurt_tz = pytz.timezone('Europe/Berlin')
            # 將 '交易開始時間' 和 '交易結束時間' 從字串轉換為 datetime 對象
            # 並轉換為當地時間
            df['交易開始時間'] = pd.to_datetime(df['交易開始時間']).dt.tz_localize(frankfurt_tz)
            df['交易結束時間'] = pd.to_datetime(df['交易結束時間']).dt.tz_localize(frankfurt_tz)
            # 轉換為台灣時間 (UTC+8)
            df['交易開始時間'] = df['交易開始時間'].dt.tz_convert(taipei_tz).dt.strftime('%Y-%m-%d %H:%M')
            df['交易結束時間'] = df['交易結束時間'].dt.tz_convert(taipei_tz).dt.strftime('%Y-%m-%d %H:%M')
            # 將這個商品的數據添加到總的 DataFrame 中
            all_data = pd.concat([all_data, df])
        except json.JSONDecodeError as e:
            print(f"JSON 無法解析 {_ISIN_CODE} 的數據： {e}")

# 儲存所有商品的數據到 Excel 文件
excel_file = 'all_price_information.xlsx'
# 檢查文件是否存在
if os.path.isfile(excel_file):
    # 讀取已有的數據
    existing_data = pd.read_excel(excel_file)
    # 將新的數據附加在已有數據後面
    all_data = pd.concat([existing_data, all_data])

# 儲存到 Excel 文件，如果文件存在則不覆蓋，而是附加數據
all_data.to_excel(excel_file, index=False)
print(f"所有商品的資料已儲存或更新至 {excel_file}")



所有商品的資料已儲存或更新至 all_price_information.xlsx


以證券編號為主索引、最後更新時間為次索引排序數據

In [1]:
# 導入庫
import requests
import json
import pandas as pd
# 台灣時區
import pytz
# 處理檔案時需要路徑
import os

# 定義要查詢的 ISIN 程式碼列表
isin_codes = [
    'US872898AJ06',
    'USF2893TAE67',
    'US02209SBE28',
    'US02209SBF92',
    'US05526DBV64',
    'US30303M8J41'
]

# 定義欄位名稱映射
column_mapping = {
    'isin': '證券編碼',
    'lastPrice': '最新價格',
    'timestampLastPrice': '最新價格時間戳',
    'changeToPrevDayAbsolute': '今日漲跌價格',
    'changeToPrevDayInPercent': '今日漲跌幅度',
    'closingPricePrevTradingDay': '前日收盤價',
    'mic': '市場識別碼',
    'dayHigh': '最高價',
    'dayLow': '最低價',
    'priceFixings': '價格定點',
    'tradedInPercent': '以百分比交易',
    'tradingTimeEnd': '交易結束時間',
    'tradingTimeStart': '交易開始時間',
    'turnoverInEur': '成交額（歐元）',
    'turnoverInPieces': '成交量（件數）',
    'turnoverNominal': '名義成交額',
    'weeks52High': '52週最高價',
    'weeks52Low': '52週最低價',
    'currency': '貨幣',
    'minimumTradableUnit': '最小可交易單位'
}

# 定義一個空的 DataFrame 來儲存所有商品的數據
all_data = pd.DataFrame()

# 遍歷 ISIN 程式碼列表：這裡我用 _ISIN_CODE 來做索引，主要是因為要寫入 URL，這樣看清楚些
for _ISIN_CODE in isin_codes:
    # API 網址
    url = f"https://api.boerse-frankfurt.de/v1/data/price_information?isin={_ISIN_CODE}&mic=XFRA"
    # 使用 with 來處理 request 可正確監督執行流程
    with requests.get(url, stream=True) as response:
        if response.status_code == 200:
            # 要捕捉 API 出現 `data:` 這個字串的次數，第二次起是重複的，就可結束資料取得
            data_count = 0
            data_json = None
            # 逐行解析
            for line in response.iter_lines():
                # 解碼
                line = line.decode('utf-8')
                if 'data:' in line:
                    data_count += 1
                    if data_count == 1:
                        # 去除空格
                        data_json = line.split('data:', 1)[1].strip()
                    elif data_count == 2:
                        break
        else:
            # 如果請求失敗
            print(f"發生錯誤：無法取得 {_ISIN_CODE} 的資料：{response.status_code}")
            # 進入下一輪 Loop 的 ISIN 查詢
            continue
    # 解析內容
    if data_json:
        try:
            # 將 JSON 字串轉換為字典
            data = json.loads(data_json)
            # 將字典轉換為 DataFrame
            df = pd.DataFrame([data])
            '''
            因為要儲存到 Excel、使用 drop_duplicates()等情境需要進行比對
            而這些操作必須確保數據具備可哈希特性
            將 DataFrame 中所有包含字典的欄位轉換為 JSON 字串
            因為 DataFrame 的欄位必須是可哈希 hashable 的數據類型
            而字典不是 hashable，轉換為字串以確保這些操作能夠正常進行
            '''
            # 遍歷所有欄位：檢查 json 中是否包含字典
            for column in df.columns:
                # 判斷欄位內容是否為字典
                if isinstance(df[column].iloc[0], dict):
                    # 將該列中的每個元素轉換為 JSON 字串
                    df[column] = df[column].apply(json.dumps)
            
            # mapping
            df.rename(columns=column_mapping, inplace=True)

            # 處理時間轉換等
            # 將 '最新價格時間戳' 轉換為台灣時間 (UTC+8) 並格式化
            df['最新價格時間戳'] = pd.to_datetime(df['最新價格時間戳'])
            taipei_tz = pytz.timezone('Asia/Taipei')
            df['最新價格時間戳'] = df['最新價格時間戳'].dt.tz_convert(taipei_tz)
            df['最新價格時間戳'] = df['最新價格時間戳'].dt.strftime('%Y-%m-%d %H:%M')
            
            # 原始數據為中歐時間（CET/CEST）
            frankfurt_tz = pytz.timezone('Europe/Berlin')
            # 將 '交易開始時間' 和 '交易結束時間' 從字串轉換為 datetime 對象
            # 並轉換為當地時間
            df['交易開始時間'] = pd.to_datetime(df['交易開始時間']).dt.tz_localize(frankfurt_tz)
            df['交易結束時間'] = pd.to_datetime(df['交易結束時間']).dt.tz_localize(frankfurt_tz)
            # 轉換為台灣時間 (UTC+8)
            df['交易開始時間'] = df['交易開始時間'].dt.tz_convert(taipei_tz).dt.strftime('%Y-%m-%d %H:%M')
            df['交易結束時間'] = df['交易結束時間'].dt.tz_convert(taipei_tz).dt.strftime('%Y-%m-%d %H:%M')
            # 將這個商品的數據添加到總的 DataFrame 中
            all_data = pd.concat([all_data, df])
        except json.JSONDecodeError as e:
            # 如果 JSON 解析失敗
            print(f"JSON 無法解析 {_ISIN_CODE} 的數據： {e}")

'''處理資料的儲存：EXCEL 部分'''
# 將取得的數據按照 '證券編碼' 和 '最新價格時間戳' 進行排序
all_data.sort_values(by=['證券編碼', '最新價格時間戳'], inplace=True)

# 檢查文件是否存在，並讀取已有數據
excel_file = 'all_price_information.xlsx'
# 檢查 Excel 檔案是否存在
if os.path.isfile(excel_file):
    # 如果存在，則讀取現有數據
    existing_data = pd.read_excel(excel_file)
    # 將新數據附加到現有數據
    combined_data = pd.concat([existing_data, all_data]).drop_duplicates(subset=['證券編碼', '最新價格時間戳'])
else:
    # 如果檔案不存在，則將 all_data 設為 combined_data
    combined_data = all_data

# 根據 '證券編碼' 和 '最新價格時間戳' 排序 combined_data
combined_data.sort_values(by=['證券編碼', '最新價格時間戳'], inplace=True)

# 儲存或更新已排序的數據到 Excel 檔案
combined_data.to_excel(excel_file, index=False)
print(f"所有商品的資料已儲存或更新至 {excel_file}")

所有商品的資料已儲存或更新至 all_price_information.xlsx
